In [14]:
import pickle
import re
import numpy as np

basis_file = '/home/anton/Documents/Tulane/Research/PNC_Good/AngleBasis1.pkl'
demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/PNC/'

basis = pickle.load(open(basis_file, 'rb'))

pnc_thetas = []
pnc_jitter = []
pnc_sex = []
pnc_race = []
pnc_fc = []

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))
pnc_subs = []
pnc_tasks = []

for subtask in basis:
    m = re.search('([^-]+)-(.*)', subtask)
    sub = m.group(1)
    task = m.group(2)
    if sub not in demo['Race'] or demo['Race'][sub] not in ['AA', 'EA']:
        continue
    s = demo['Sex'][sub] == 'M'
    r = demo['Race'][sub]
    pnc_subs.append(sub)
    pnc_tasks.append(task)
    pnc_sex.append(s)
    pnc_race.append(r == 'AA')
    pnc_thetas.append(basis[subtask]['thetas'])
    pnc_jitter.append(basis[subtask]['jitter'])
    p = np.load(f'{demodir}/fc/{sub}_task-{task}_fc.npy')
    pnc_fc.append(p)
    
pnc_thetas = np.stack(pnc_thetas)
pnc_jitter = np.stack(pnc_jitter)
pnc_sex = np.array(pnc_sex).astype('int')
pnc_race = np.array(pnc_race).astype('int')
pnc_fc = np.stack(pnc_fc)

print([a.shape for a in [pnc_thetas, pnc_jitter, pnc_sex, pnc_race, pnc_fc]])

[(3849, 1, 264), (3849, 1, 264), (3849,), (3849,), (3849, 34716)]


In [15]:
import pickle
import re
import numpy as np

basis_file = '/home/anton/Documents/Tulane/Research/BSNIP/AngleBasis1.pkl'
demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/BSNIP/'

basis = pickle.load(open(basis_file, 'rb'))

bsnip_thetas = []
bsnip_jitter = []
bsnip_fc = []
bsnip_race = []
bsnip_sex = []

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))
bsnip_subs = []

for subtask in basis:
    m = re.search('([^-]+)', subtask)
    sub = m.group(1)
    s = demo['sex'][sub] == 's1.0' # Male
    r = demo['Race'][sub]
    if r not in ['AA', 'CA']:
        continue
    bsnip_sex.append(s)
    bsnip_race.append(r == 'AA')
    bsnip_subs.append(sub)
    bsnip_thetas.append(basis[subtask]['thetas'])
    bsnip_jitter.append(basis[subtask]['jitter'])
    p = np.load(f'{demodir}/fc/{sub}_task-unk_fc.npy')
    bsnip_fc.append(p)
    
bsnip_thetas = np.stack(bsnip_thetas)
bsnip_jitter = np.stack(bsnip_jitter)
bsnip_fc = np.stack(bsnip_fc)
bsnip_sex = np.array(bsnip_sex).astype('int')
bsnip_race = np.array(bsnip_race).astype('int')

print([a.shape for a in [bsnip_thetas, bsnip_jitter, bsnip_fc, bsnip_sex, bsnip_race]])

[(1165, 1, 264), (1165, 1, 264), (1165, 34716), (1165,), (1165,)]


In [9]:
def rmse(yhat, y):
    if isinstance(yhat, np.ndarray) or isinstance(yhat, int):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

def tops(thetas, jitter):
    t0 = np.expand_dims(thetas, 2)
    t1 = np.expand_dims(thetas, 3)
    j0 = np.expand_dims(jitter, 2)
    j1 = np.expand_dims(jitter, 3)
    ps = np.cos(t0-t1)*(j0*j1)
    a,b = np.triu_indices(264, 1)
    ps = ps[:,:,a,b]
    return ps

def tops2(thetas, jitter, fc):
    paps = []
    pres = []

    for i in range(0,fc.shape[0],500):
        ps = tops(thetas[i:i+500], jitter[i:i+500])
        aps = np.mean(ps, axis=1)
        res = fc[i:i+500] - aps
        paps.append(aps)
        pres.append(res)
        print(f'Done {i}')

    aps = np.concatenate(paps)
    res = np.concatenate(pres)
    
    return aps, res

print('Complete')

Complete


In [11]:
# AB20

pnc_ab20, pnc_res20 = tops2(pnc_thetas, pnc_jitter, pnc_fc)

print(pnc_ab20.shape)
print(pnc_res20.shape)

bsnip_ab20, bsnip_res20 = tops2(bsnip_thetas, bsnip_jitter, bsnip_fc)

print(bsnip_ab20.shape)
print(bsnip_res20.shape)

Done 0
Done 500
Done 1000
Done 1500
Done 2000
Done 2500
Done 3000
Done 3500
(3849, 34716)
(3849, 34716)
Done 0
Done 500
Done 1000
(1165, 34716)
(1165, 34716)


In [16]:
# AB1

pnc_ab1, pnc_res1 = tops2(pnc_thetas, pnc_jitter, pnc_fc)

print(pnc_ab1.shape)
print(pnc_res1.shape)

bsnip_ab1, bsnip_res1 = tops2(bsnip_thetas, bsnip_jitter, bsnip_fc)

print(bsnip_ab1.shape)
print(bsnip_res1.shape)

Done 0
Done 500
Done 1000
Done 1500
Done 2000
Done 2500
Done 3000
Done 3500
(3849, 34716)
(3849, 34716)
Done 0
Done 500
Done 1000
(1165, 34716)
(1165, 34716)


In [12]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

def tonp(x):
    return x.detach().cpu().numpy()

def totorch(x):
    return torch.from_numpy(x).float().cuda()

def totorchidcs(x):
    return torch.from_numpy(x).long().cuda() #F.one_hot(torch.from_numpy(x)).float().cuda()

def rmse(yt, yhat):
    return torch.mean((yt-yhat)**2)**0.5

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(34716,100).float().cuda()
        self.l2 = nn.Linear(100,2).float().cuda()
        
    def forward(self, x):
        x = F.relu(self.l1(x))
        x = self.l2(x)
        return x
    
def fit_mlp(xtr, ytr, verbose=False):
    xtr = totorch(xtr)
    ytr = totorchidcs(ytr)
    
    ce = nn.CrossEntropyLoss()
    mlp = MLP()
    optim = torch.optim.Adam(mlp.parameters(), lr=5e-4, weight_decay=5e-4)

    nepochs = 1000
    pperiod = 100

    for epoch in range(nepochs):
        optim.zero_grad()
        yhat = mlp(xtr)
        loss = ce(yhat, ytr)
        loss.backward()
        optim.step()
        if verbose:
            if epoch % pperiod == 0 or epoch == nepochs-1:
                print(f'{epoch} {float(loss)}')

    if verbose:
        print('Complete')
    
    return mlp
    
def predict_help(model, xt):
    xt = totorch(xt)
    with torch.no_grad():
        yhat = model(xt)
        return tonp(yhat)
    
print('Done')

Done


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.metrics import roc_auc_score

def cat(x, **kwargs):
    return np.concatenate(x, **kwargs)

def rmse(yhat, y):
    if isinstance(yhat, np.ndarray) or isinstance(yhat, int):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

def predict(xtr, xt, ytr, yt, lst):
    reg = fit_mlp(xtr, ytr)
    p = predict_help(reg, xt)
    acc = roc_auc_score(yt, p[:,1])
    print(acc)
    lst.append(acc)
    return p
    
def get_res(fctr, fct, abtr, abt):
    return fctr-abtr, fct-abt

def combine(yt, p0, p1, lst):
    p = (p0+p1)/2
    acc = roc_auc_score(yt, p[:,1])
    print(acc)
    lst.append(acc)

rfc = []
rfc2 = []
rfc_ens = []

rab20 = []
rres1 = []
rbest = []

x1a = bsnip_fc
x1b = pnc_fc
x2a = bsnip_ab20
x2b = pnc_ab20
x3a = bsnip_res1
x3b = pnc_res1
ya = bsnip_sex
yb = pnc_sex

for i in range(10):

    p1 = predict(x1a, x1b, ya, yb, rfc)
    p2 = predict(x1a, x1b, ya, yb, rfc)
    combine(yb, p1, p2, rfc_ens)
    
    p1 = predict(x2a, x2b, ya, yb, rab20)
    p2 = predict(x3a, x3b, ya, yb, rres1)
    combine(yb, p1, p2, rbest)
    
    print('---')
    
print(np.mean(rfc), np.std(rfc))
print(np.mean(rfc_ens), np.std(rfc_ens))
print(np.mean(rab20), np.std(rab20))
print(np.mean(rres1), np.std(rres1))
print(np.mean(rbest), np.std(rbest))

0.7213427851534008
0.7166706647916093
0.7198197973789989
0.7073134279056847
0.7261364737837623
0.7514142802914823
---
0.7028198693993986
0.7200984026091962
0.7167884425128297
0.6952592165805039
0.7264318657236507
0.7376721923821912
---
0.6836154673826107
0.7084683264570999
0.7228998878539491
0.7104243842661961
0.7264963050285942
0.751885661929746
---
0.716242468317793
0.7099900958412824
0.7166663327374954
0.7147155546193504
0.7261929258639334
0.7575632249760247
---
0.6975952767613997
0.7172200234039223
0.7117345598822548
0.7013083886437368
0.7264334902439433
0.7477203918992754
---
0.6837825222193763
0.6974477161681465
0.7204105812587758
0.6833487753012266
0.7262593958192428
0.7366823180171755
---
0.700488818156072
0.7117280618010839
0.7100169004261117
0.7147843259784079
0.7260365657857615
0.7550810933454776
---
0.7159932398295553
0.719763751428901
0.7180607126553921
0.685901167434433
0.7267776177926126
0.7322416917970931
---
0.7188693176310812
0.7202261982055549
0.7195452534495335
0.69